In [1]:
import time
from ortools.sat.python import cp_model

# def read_data():
#     # Input data
#     N, M = map(int, input().split())
#     d = list(map(int, input().split()))
#     c = list(map(int, input().split()))
#     K = int(input())

#     conflict = []

#     for k in range(K):
#         i, j = map(int, input().split())
#         conflict.append((i - 1, j - 1))
#         conflict.append((j - 1, i - 1))

#     return N, M, d, c, conflict


def read_data(input_file):
    """
    Read data from stdin
    """
    with open(input_file, 'r') as f:
        lines = f.readlines()
        N, M = list(map(int, lines[0].split()))
        d = list(map(int, lines[1].split()))
        assert len(d) == N, "Number of classes does not match"
        c = list(map(int, lines[2].split()))
        assert len(c) == M, "Number of rooms does not match"
        k = int(lines[3])
        conflict = []
        for i in range(4, 4 + k):
            s1, s2 = list(map(int, lines[i].split()))
            conflict.append([s1 - 1, s2 - 1])
    return N, M, d, c, conflict



class CP:
    def __init__(self):
        pass

    def solve(self, N, M, d, c, conflict):
        # Create the CP model
        model = cp_model.CpModel()

        # Decision variables
        s = [model.NewIntVar(0, N - 1, "s[%i]" % i) for i in range(N)]
        y = [[model.NewIntVar(0, 1, "y[%i][%i]" % (i, j)) for j in range(M)] for i in range(N)]
        timeslot = model.NewIntVar(0, N - 1, "timeslot")

        # Constraints
        for i, j in conflict:
            model.Add(s[i] != s[j])

        for i in range(N):
            model.Add(timeslot >= s[i])

        for i in range(N):
            model.Add(sum(y[i][j] * c[j] for j in range(M)) >= d[i])

        for i in range(N):
            for j in range(N):
                if i == j:
                    continue
                for k in range(M):
                    b = model.NewBoolVar("b[%i][%i]" % (i, j))
                    model.Add(s[i] == s[j]).OnlyEnforceIf(b)
                    model.Add(s[i] != s[j]).OnlyEnforceIf(b.Not())
                    model.Add(y[i][k] + y[j][k] <= 1).OnlyEnforceIf(b)

        for i in range(N):
            model.Add(sum([y[i][j] for j in range(M)]) == 1)

        # Objective function (Minimize the timeslot)
        model.Minimize(timeslot)

        # Create a solver
        solver = cp_model.CpSolver()

        # Solve
        solver.Solve(model)
        day = round(solver.Value(timeslot)) // 4 + 1

        for i in range(N):
            room = -1
            for j in range(M):
                if solver.Value(y[i][j]) == 1:
                    room = j
                    break
            print(f"{i + 1} {solver.Value(s[i]) % 4 + 1} {room + 1}")

        return day, N, M, d, c, conflict


In [2]:
import os
import time
import pandas as pd

test_cases = []
num_subjects = []
num_rooms = []
num_conflicts = []
times = []
objective_values = []

solver = CP()

for test_case in [
    # 'data_10_2_(0).txt', 'data_10_2_(1).txt', 'data_10_2_(2).txt', 
    'data_16_3_(2).txt',
    # 'data_20_4_(0).txt', 'data_20_4_(1).txt', 'data_20_4_(2).txt',
    # 'data_16_3_(1).txt',
    # 'data_16_3_(0).txt',  
    ]:

    print(f"Start solving {test_case}")
    start_time = time.time()
    N, M, d, c, conflict = read_data("../data/" + test_case)
    day, N, M, d, c, conflict = solver.solve(N, M, d, c, conflict)
    end_time = time.time()
    test_cases.append(test_case)
    num_subjects.append(N)
    num_rooms.append(M)
    num_conflicts.append(len(conflict))
    times.append(end_time - start_time)
    objective_values.append(day)
    print(f"Finished solving {test_case}")
    print(f"Time: {end_time - start_time}")

result = pd.DataFrame({
    "test_case": test_cases,
    "num_subjects": num_subjects,
    "num_rooms": num_rooms,
    "num_conflicts": num_conflicts,
    "time": times,
    "objective_value": objective_values
})

Start solving data_16_3_(2).txt
